# GPT-OSS Legal Contract Summarizer

This notebook demonstrates how to run OpenAI's GPT‑OSS models (20B or 120B) for summarizing legal contracts and flagging compliance risks.

It supports TXT, CSV, and PDF input formats.

⚠️ **Note:** GPT‑OSS‑120B requires substantial GPU memory (8×A100 80GB). For Colab, use GPT‑OSS‑20B with `load_in_8bit=True` if you encounter out-of-memory errors.

In [1]:
# Step 0 — Install latest dependencies
!pip install --upgrade --quiet transformers accelerate bitsandbytes sentencepiece pandas pypdf
!git lfs install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 15.6 MB/s eta 0:00:00
Git LFS initialized.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from pypdf import PdfReader

In [3]:
contract_text = """
Section 14: Data Protection Obligations
The Vendor shall comply with all applicable data-protection laws, including GDPR and CCPA.
The Vendor must notify the Client within 72 hours of any data breach.
The Vendor shall not transfer Client Data outside approved jurisdictions without prior written consent.

Section 18: Termination Rights
The Client may terminate this Agreement with 30 days' written notice if the Vendor breaches compliance obligations.
"""

In [4]:
model_name = "openai/gpt-oss-20b"  # Change to gpt-oss-120b for higher capacity

In [5]:
print(f"Loading model: {model_name} ... This may take a few minutes.")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=False
)

Loading model: openai/gpt-oss-20b ... This may take a few minutes.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

ValueError: The checkpoint you are trying to load has model type `gpt_oss` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [ ]:
prompt = f"""
You are a legal compliance assistant.
Summarize the key obligations and termination rights from the following contract.
Also list any potential compliance risks.

Contract:
{contract_text}
"""

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=400,
    temperature=0.2,
    do_sample=False
)
print("\n" + "="*30 + " MODEL OUTPUT " + "="*30)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
!pip install --upgrade transformers